# Hitmaps and White noise covariance
> Load and scale hitmaps and white noise covariance matrices
- toc: True

* Basically the same scaling of the noise

In [ ]:
# default_exp hitmap_wcov

In [ ]:
import logging as log

In [ ]:
# export

import healpy as hp
import numpy as np
from pathlib import Path
import logging as log

from s4_design_sim_tool.core import (
    get_telescope,
    get_band,
    get_observing_efficiency,
    base_folder,
    simulations_observing_efficiency,
)
from s4_design_sim_tool.noise import get_thinfp, get_tube_years

Simulations are executed at lower sampling frequency:

* $20~Hz$ for SAT (production is $100~Hz$)
* $200~Hz$ for LAT (production is $400~Hz$)

In [ ]:
# export

simulations_sampling_frequency_scaling = dict(SAT=5, LAT=2)

In [ ]:
# exports


def load_hitmap_wcov(config, site, channel, realization=0):
    """Load hitmaps and white noise covariance matrices for a channel

    This loads the simulated hitmaps and white noise covariance matrices
    and scales them properly to the experiment configuration and duration
    as defined in the input config file.
    Hitmaps assumes a sampling frequency of 100 Hz for SAT and 400 Hz for
    LAT.

    Parameters
    ----------
    config : dict
        CMB-S4 configuration,
        generally loaded from a TOML file
    site : str
        'Pole' or 'Chile', case doesn't matter
    channel : str
        Channel tag, e.g. 'MFHS1'
    realization : int
        Choose one of the available 8 realizations

    Returns
    -------
    hitmap : numpy array
        Hitmap for all channels all tubes combined
    wcov : numpy array
        White noise covariance matrix, rows are:
        "II", "IQ", "IU", "QQ", "QU", "UU", units are K^2
    """

    # it is the same scaling for hitmap and white noise covariance matrix,
    # which is the same as noise except squared

    telescope = get_telescope(channel)
    band = get_band(channel)

    scaling = (
        365.25
        * get_observing_efficiency(
            config["experiment"]["observing_efficiency"], site, telescope, band
        )
    ) / (10 * simulations_observing_efficiency[site.lower()].get(telescope, 1))
    scaling *= get_tube_years(config, site, channel)
    # focal plane thinning factor of TOD simulations
    scaling *= get_thinfp(channel)

    map_filename = (
        Path(f"{realization:08d}")
        / f"{site.lower()}_noise_{telescope}_{channel}_filtered_telescope_all_time_all_hmap.fits"
    )
    wcov_filename = (
        Path(f"{realization:08d}")
        / f"{site.lower()}_noise_{telescope}_{channel}_filtered_telescope_all_time_all_wcov.fits"
    )
    log.info(f"Base folder: {base_folder}")
    log.info(f"Reading {map_filename}")
    hitmap = hp.read_map(Path(base_folder) / map_filename, 0, dtype=None, verbose=False)
    wcov = hp.read_map(
        Path(base_folder) / wcov_filename, (0, 1, 2, 3, 4, 5), dtype=None, verbose=False
    )
    hitmap[hitmap == hp.UNSEEN] = 0
    # input map is 10 days at 100% efficiency
    hitmap = np.round(hitmap * scaling).astype(np.int64)
    hitmap *= simulations_sampling_frequency_scaling[telescope]
    wcov /= scaling
    wcov[:, hitmap == 0] = hp.UNSEEN
    return hitmap, wcov

## Available input maps

In [ ]:
import toml

config = toml.load("s4_reference_design.toml")

In [ ]:
filenames = !ls $base_folder/00000000/*noise*wcov*

In [ ]:
import os.path
for f in map(os.path.basename, filenames):
    print(f)

In [ ]:
channel = "HFS1"
site = "Pole"

In [ ]:
telescope = get_telescope(channel)

In [ ]:
input_map = hp.read_map(
    base_folder + "/00000000/" + \
    f"{site.lower()}_noise_{telescope}_{channel}_filtered_telescope_all_time_all_hmap.fits"
, 0)

In [ ]:
from astropy.io import fits

In [ ]:
fits.open(base_folder + "/00000000/" + \
    f"{site.lower()}_noise_{telescope}_{channel}_filtered_telescope_all_time_all_wcov.fits"
)[1].header

In [ ]:
input_EE_wcov = hp.read_map(base_folder + "/00000000/" + \
    f"{site.lower()}_noise_{telescope}_{channel}_filtered_telescope_all_time_all_wcov.fits"
, (3))

In [ ]:
hitmap, wcov = load_hitmap_wcov(config, site, channel, realization=0)

In [ ]:
num_tubes = sum([
    telescope[channel[:-1]] for telescope in config["telescopes"][telescope].values()
])

In [ ]:
sampling_rate = input_map.sum() 
sampling_rate /= 10 * 24 * 3600 * 0.3931149305555555
sampling_rate /= 8438 # HF detectors per tube
sampling_rate *= 8 # focal plane thinning factor

In [ ]:
sampling_rate

Check we are within 5% of 20 Hz (used in sims for SAT, LAT used 200 Hz)

In [ ]:
np.testing.assert_allclose(sampling_rate, 20, rtol=5/100)

In [ ]:
import astropy.units as u

In [ ]:
input_variance = input_EE_wcov * u.K**2 * (input_map  / (20 * u.Hz))

In [ ]:
input_variance = input_variance[input_map > 0]

In [ ]:
input_NET = np.sqrt(np.median(input_variance))

In [ ]:
input_NET.to(u.uK * u.s**(.5))

In [ ]:
input_NET.to(u.uK * u.s**(.5)) / np.sqrt(2)

In [ ]:
from astropy.tests.helper import assert_quantity_allclose

In [ ]:
channel_NET = 747 * u.uK * u.s**(.5)

The channel NET is valid for temperature, when we solve for the $2$ polarization components we get a factor of $\sqrt{2}$.

In [ ]:
assert_quantity_allclose(input_NET, channel_NET*np.sqrt(2), rtol=5/100) 

In [ ]:
sampling_rate = hitmap.sum() / (config["experiment"]["total_experiment_length_years"] * config["experiment"]["observing_efficiency"]["default"])
sampling_rate /= 365.25 * 24 * 3600
sampling_rate /= 8438 # HF detectors per tube
sampling_rate /= num_tubes

In [ ]:
sampling_rate

In [ ]:
np.testing.assert_allclose(sampling_rate, 100, rtol=5/100)

In [ ]:
# Compare mask
np.testing.assert_allclose(input_map == 0, hitmap == 0)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
log_hitmap = np.log10(hitmap)

In [ ]:
log_hitmap[np.isinf(log_hitmap)] = hp.UNSEEN

In [ ]:
hp.mollview(log_hitmap, unit="log10(samples)", min=3, max=8.5, title="Hitmap")

In [ ]:
for i,label in [(3, "EE"), (4, "EB"), (5, "BB")]:
    hp.mollview(wcov[i], unit="$K^2_{CMB}$", min=0, max=1e-11, title=f"{label} White noise covariance")

In [ ]:
wcov[wcov == hp.UNSEEN] = 0

In [ ]:
variance = wcov[3] * u.K**2 * (hitmap / (100 * u.Hz)) 

In [ ]:
variance[variance != 0].min(), variance.max(), variance[variance != 0].mean()

In [ ]:
full_mission_NET = np.sqrt(np.median(variance[hitmap != 0]))

In [ ]:
full_mission_NET.to(u.uK * u.s**(.5))

In [ ]:
assert_quantity_allclose(full_mission_NET, channel_NET*np.sqrt(2), rtol=5/100) 